In [1]:
import pandas as pd

df = pd.read_csv('Sample - Superstore.csv')

df.head() 

# Check that csv file has been read properly

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [2]:
# Profit over time

datesAndProfit = df[['Ship Date', 'Profit']]

sorted_datesAndProfit = datesAndProfit.sort_values(by='Ship Date')

sorted_datesAndProfit.head()



,Ship Date,Profit
8671,1/1/2015,-25.1090
8968,1/1/2015,-8.8038
8670,1/1/2015,-18.2352
8804,1/1/2015,8.6940
7343,1/1/2015,19.1880


In [3]:
# Sales over time 

salesVsTime = df[['Quantity', 'Ship Date']]

salesVsTime = datesAndProfit.sort_values(by='Ship Date')

In [4]:
# Gross sales vs Profit

In [5]:
# Sales by Region

In [6]:
# Sales by State

In [7]:
# Sales by City

In [8]:
# Sales by segment - Consumer, corporate, home office, etc

In [9]:
# Average profit per item sold by segment

In [10]:
# Number of sales by category

In [11]:
# Gross sales vs profit for each category

In [12]:
# Number of days to deliver for standard class

In [13]:
# Number of days to deliver for second class